# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

# 1 $n$-gram Language Model

In [2]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/test.txt

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## 1.1 Building vocabulary

### Code

In [3]:
with open('data/lm/train.txt', 'r', encoding="utf-8") as f:
    train = dict()
    for line in f:
        line = '<START> ' + line + ' <END>'     # to add <START> and <END> to the start and end of each sentence 
        tmp = line.split()
        for word in tmp:
            if word in train.keys():            # if in dict, increment counter
                train[word] += 1
            else:
                train[word] = 1                 # if not in dict, add new entry

    countUNK = 0
    for key in list(train.keys()):              # find all words with freq < 3, sum their counts
        if train[key] < 3:
            countUNK += train[key]
            del train[key]
    train['<UNK>'] = countUNK
    print("Size of vocabulary = " + str(len(train)))

Size of vocabulary = 26601


### Discussion

Size of vocabulary = 26601

Number of parameters of n-gram models = (size of vocabulary) ^ n = (26601) ^ n
eg unigram: 26601 ^ 1
eg bigram: 26601 ^ 2
eg trigram: 26601 ^ 3

## 1.2 $n$-gram Language Modeling

### Code

In [111]:
import math

unigramDict = dict()
bigramDict = dict()
bigramTrain = dict()

# build unigram
def unigram():
    for key in train.keys():
        unigramDict[key] = float(train[key]) / sum(train.values())

def unigramProb(word):
    if word in unigramDict.keys():
        return unigramDict[word]
    else:
        return unigramDict['<UNK>']

# build bigram
def bigramVocab():
    with open('data/lm/train.txt', 'r', encoding="utf-8") as f:
        for line in f:
            line = '<START> ' + line + ' <END>'
            tmp = line.split()
            for i in range(len(tmp) - 1):
                if tmp[i] not in train.keys():
                    tmp[i] = '<UNK>'
                if tmp[i+1] not in train.keys():
                    tmp[i+1] = '<UNK>'
                    
                if (tmp[i], tmp[i+1]) in bigramTrain.keys():
                    bigramTrain[(tmp[i], tmp[i+1])] += 1
                else:
                    bigramTrain[(tmp[i], tmp[i+1])] = 1

def bigram():
    for key in bigramTrain.keys():
        bigramDict[key] = float(bigramTrain[key]) / train[key[0]]    # P(w2|w1) = C(w1, w2) / C(w1)
           
unigram()
bigramVocab()
bigram()


In [113]:
def unigramPerplexity(set):
    with open(f'data/lm/{set}.txt', 'r', encoding="utf-8") as f:
        N = 0
        sum = 0
        for line in f:
            line = '<START> ' + line + ' <END>'
            tmp = line.split()
            # print(tmp)
            logProb = 1
            for word in tmp:
                N += 1
                # logProb *= unigramProb(word)
                logProb += math.log2(unigramProb(word))
                # print(logProb, unigramProb(word), word, line)
            sum += (logProb)
    return 2 ** (-sum / N)

def bigramPerplexity(set):
    with open(f'data/lm/{set}.txt', 'r', encoding="utf-8") as f:
        N = 0
        sum = 0
        for line in f:
            line = '<START> ' + line + ' <END>'
            tmp = line.split()
            logProb = 1
            for i in range(len(tmp) - 1):
                N += 1
                if tmp[i] not in train.keys():
                    tmp[i] = '<UNK>'
                if tmp[i+1] not in train.keys():
                    tmp[i+1] = '<UNK>'
                    
                if (tmp[i], tmp[i+1]) not in bigramDict.keys():
                    prob = 0 / train[key[0]]
                else:
                    prob = bigramDict[(tmp[i], tmp[i+1])]
                # logProb += math.log2(bigramDict[(tmp[i], tmp[i+1])])
                logProb += math.log2(prob)
            sum += (logProb)
    return 2 ** (-sum / N)

print("Unigram train perplexity = " + str(unigramPerplexity("train")))
print("Unigram dev perplexity = " + str(unigramPerplexity("dev")))
print("Bigram train perplexity = " + str(bigramPerplexity("train")))
print("Bigram dev perplexity = " + str(bigramPerplexity("dev")))


Unigram train perplexity = 866.6059171242847
Unigram dev perplexity = 795.3775132279369
Bigram train perplexity = 74.93908934229735


ValueError: math domain error

### Discussion

When calculating the perplexity of unigram on the dev set, even if the words aren't in the training set, they are counted as \<UNK\>, a valid token in the vocabulary.
However, when it comes to the bigram model, some new combinations of words are in the dev set does not exist in the training set. These combinations have a count and probability of zero, making the log likelihood of the model negative infinity which caused error when calculating the perplexity.

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

In [117]:
bigramTrainDict = dict()
bigramSmoothDict = dict()

def bigramTrain():
    with open('data/lm/train.txt', 'r', encoding="utf-8") as f:
        for line in f:
            line = '<START> ' + line + ' <END>'
            tmp = line.split()
            for i in range(len(tmp) - 1):
                if tmp[i] not in train.keys():
                    tmp[i] = '<UNK>'
                if tmp[i+1] not in train.keys():
                    tmp[i+1] = '<UNK>'
                    
                if (tmp[i], tmp[i+1]) in bigramTrainDict.keys():
                    bigramTrainDict[(tmp[i], tmp[i+1])] += 1
                else:
                    bigramTrainDict[(tmp[i], tmp[i+1])] = 1

def bigramSmoothPerplexity(set, k):
    for key in bigramTrainDict.keys():
        bigramSmoothDict[key] = (float(bigramTrainDict[key]) + k) / (train[key[0]] + k*26601)    # P(w2|w1) = (C(w1, w2) + 1)/ (C(w1) + |V|)
    
    with open(f'data/lm/{set}.txt', 'r', encoding="utf-8") as f:
        N = 0
        sum = 0
        for line in f:
            line = '<START> ' + line + ' <END>'
            tmp = line.split()
            logProb = 1
            for i in range(len(tmp) - 1):
                N += 1
                if tmp[i] not in train.keys():
                    tmp[i] = '<UNK>'
                if tmp[i+1] not in train.keys():
                    tmp[i+1] = '<UNK>'

                if (tmp[i], tmp[i+1]) not in bigramSmoothDict.keys():
                    prob = (k) / (train[key[0]] + k*26601)
                else:
                    prob = bigramSmoothDict[(tmp[i], tmp[i+1])]
                    
                logProb += math.log2(prob)
            sum += (logProb)
    return 2 ** (-sum / N)

bigramTrain()
print("Bigram train perplexity = " + str(bigramSmoothPerplexity("train", 1)))
print("Bigram dev perplexity = " + str(bigramSmoothPerplexity("dev", 1)))


Bigram train perplexity = 1405.177331370177
Bigram dev perplexity = 1580.6409074084834


#### Discussion

#### 1.3.2 Add-k smoothing

##### Code

In [118]:
print("Bigram train perplexity (k=0.001) = " + str(bigramSmoothPerplexity("train", 0.001)))
print("Bigram train perplexity (k=0.01) = " + str(bigramSmoothPerplexity("train", 0.01)))
print("Bigram train perplexity (k=0.1) = " + str(bigramSmoothPerplexity("train", 0.1)))
print()
print("Bigram dev perplexity (k=0.001) = " + str(bigramSmoothPerplexity("dev", 0.001)))
print("Bigram dev perplexity (k=0.01) = " + str(bigramSmoothPerplexity("dev", 0.01)))
print("Bigram dev perplexity (k=0.1) = " + str(bigramSmoothPerplexity("dev", 0.1)))


Bigram train perplexity (k=0.001) = 92.49453763806177
Bigram train perplexity (k=0.01) = 153.61489113220662
Bigram train perplexity (k=0.1) = 396.8822034902569

Bigram dev perplexity (k=0.001) = 432.7520905911309
Bigram dev perplexity (k=0.01) = 395.3147101966442
Bigram dev perplexity (k=0.1) = 631.6455075022197


##### Discussion

### 1.3.3 Linear Interpolation

#### Code

#### Discussion

##### 1.3.4 Optimization

#### Discussion

# 2 Word Vectors

In [6]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

Loaded vocab size 400000


## 2.1 Find most similar word
Use cosine similarity to find the most similar word to each of these words. Report the most similar word and its cosine similarity.

In [7]:
import numpy as np
from numpy.linalg import norm

wordList = ["dog", "whale", "before", "however", "fabricate"]
wordDict = {}
vecKeys = wv_from_bin.index_to_key
for word in wordList:
  sim = -1
  most = ""
  for i in range(len(wv_from_bin)):
    x = wv_from_bin[word]
    cosine = np.dot(x,wv_from_bin[i])/(norm(x)*norm(wv_from_bin[i]))
    curword = vecKeys[i]
    if cosine >= sim and curword != word:
      sim = cosine
      most = curword
  wordDict[word] = [most, sim]
print("Answer format: {word: [most similar word, cosine similarity]}")
print(wordDict)

Answer format: {word: [most similar word, cosine similarity]}
{'dog': ['dogs', 0.81368625], 'whale': ['whales', 0.7918058], 'before': ['after', 0.8931248], 'however': ['although', 0.9336163], 'fabricate': ['fabricating', 0.61840194]}


## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

In [8]:
# dog-puppy+cat
wordList = [["dog", "puppy", "cat"], ["speak", "speaker", "sing"], ["france", "french", "england"], ["france", "wine", "england"]]

vecKeys = wv_from_bin.index_to_key
for analogy in wordList:
  targetVec = wv_from_bin[analogy[0]] - wv_from_bin[analogy[1]] + wv_from_bin[analogy[2]]
  
  candidateList = []
  similarityList = []
  for i in range(len(wv_from_bin)):
    cosine = np.dot(targetVec,wv_from_bin[i])/(norm(targetVec)*norm(wv_from_bin[i]))
    curword = vecKeys[i]
    
    if len(similarityList) < 3 or (len(similarityList) >= 3 and cosine >= similarityList[2]):
      similarityList.append(cosine) 
      similarityList.sort(reverse=True)
      candidateList.insert(similarityList.index(cosine), curword)
  print(f"{analogy[0]} : {analogy[1]} :: {analogy[2]} : ?:", candidateList[0:3])
  print("corresponding similarities:", similarityList[0:3])
# targetVec = wv_from_bin["dog"] - wv_from_bin["puppy"] + wv_from_bin["cat"]
# print("kitty", np.dot(targetVec,wv_from_bin["kitty"])/(norm(targetVec)*norm(wv_from_bin["kitty"])))
# print("kitten", np.dot(targetVec,wv_from_bin["kitten"])/(norm(targetVec)*norm(wv_from_bin["kitten"])))
# print("dogs", np.dot(targetVec,wv_from_bin["dogs"])/(norm(targetVec)*norm(wv_from_bin["dogs"])))

dog : puppy :: cat : ?: ['cat', 'dog', 'dogs']
corresponding similarities: [0.8246017, 0.745641, 0.6278181]
speak : speaker :: sing : ?: ['sing', 'speak', 'singing']
corresponding similarities: [0.7365669, 0.55140996, 0.548521]
france : french :: england : ?: ['england', 'wales', 'ireland']
corresponding similarities: [0.837254, 0.6498687, 0.6262423]
france : wine :: england : ?: ['england', 'france', 'britain']
corresponding similarities: [0.69291466, 0.6897908, 0.61666137]


# 3 Sentiment analysis

In [9]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## 3.1 Logistic Regression

### Unigram Features

### Bigram Features

### GloVe Features

Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |           |        |          |
| bigram      |           |        |          |
| GloVe       |           |        |          |

## 3.2 Better Feature